In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

print("1. Chargement du jeu de données IMDB...")
dataset = load_dataset("imdb")

print(f"Dataset chargé. Structure : {dataset}")
print(f"Exemple de données d'entraînement : {dataset['train'][0]}")

# utilisation d'un sous ensemble de données vu que c'est juste un POC
train_size = 5000
test_size = 1000
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(train_size))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(test_size))
print(f"Dataset réduit à {len(dataset['train'])} exemples d'entraînement et {len(dataset['test'])} de test.")


/opt/anaconda3/envs/projet8/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/projet8/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/projet8/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /opt/anaconda3/envs/projet8/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/projet8/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/projet8/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/projet8/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no su

1. Chargement du jeu de données IMDB...
Dataset chargé. Structure : DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
Exemple de données d'entraînement : {'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain politica

In [2]:
# 2. Prétraitement des données avec un Tokenizer de Transformer
print("\n2. Prétraitement des données (tokenization)...")
# Chargement du tokenizer pour DistilBERT (un modèle léger et rapide)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    # Applique la tokenization et le padding/truncation
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Appliquer la fonction de tokenization à l'ensemble du dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print("Tokenization terminée.")

# Définition de la classe pour les labels
# Dans IMDB, 0 est négatif, 1 est positif
num_labels = 2
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# 3. Fine-tuning du modèle Transformer
print("\n3. Fine-tuning du modèle DistilBERT...")
# Chargement du modèle pré-entraîné pour la classification de séquences
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id
)

# Définition des arguments d'entraînement
# Ces paramètres sont optimisés pour la rapidité du PoC
training_args = TrainingArguments(
    output_dir="./results",               # Répertoire pour sauvegarder les modèles et résultats
    eval_strategy="epoch",          # Évaluer à chaque fin d'epoch
    learning_rate=2e-5,                   # Taux d'apprentissage
    per_device_train_batch_size=8,        # Taille du batch par GPU/CPU pour l'entraînement
    per_device_eval_batch_size=8,         # Taille du batch par GPU/CPU pour l'évaluation
    num_train_epochs=3,                   # Nombre d'epochs (généralement 3-5 est suffisant pour le fine-tuning)
    weight_decay=0.01,                    # Décroissance du poids (régularisation)
    logging_dir='./logs',                 # Répertoire pour les logs (TensorBoard)
    logging_steps=100,                    # Fréquence de log
    save_strategy="epoch",                # Sauvegarder à chaque fin d'epoch
    load_best_model_at_end=True,          # Charger le meilleur modèle à la fin
    metric_for_best_model="accuracy",     # Mettre en œuvre le meilleur modèle en fonction de la précision
    report_to="none"                      # Désactiver les intégrations de rapport
)


2. Prétraitement des données (tokenization)...
Tokenization terminée.

3. Fine-tuning du modèle DistilBERT...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Définition de la métrique d'évaluation (Accuracy)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/var/folders/fk/s_s6rm0x0216j6tz5mvl3tpm0000gp/T/ipykernel_2175/1169984745.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
# Lancement de l'entraînement
trainer.train()
print("\nFine-tuning terminé.")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.342100,0.334177,0.894000
2,0.201500,0.367176,0.902000
3,0.088600,0.436976,0.901000



Fine-tuning terminé.


In [7]:
# 4. Évaluation du modèle fine-tuné
print("\n4. Évaluation du modèle fine-tuné...")
results = trainer.evaluate()
print(f"Résultats de l'évaluation : {results}")


4. Évaluation du modèle fine-tuné...


Résultats de l'évaluation : {'eval_loss': 0.3671758472919464, 'eval_accuracy': 0.902, 'eval_runtime': 23.3431, 'eval_samples_per_second': 42.839, 'eval_steps_per_second': 5.355, 'epoch': 3.0}


In [8]:
# 5. Tester une prédiction simple
print("\n5. Test de prédiction sur une nouvelle phrase :")
text_example_positive = "This movie was absolutely fantastic! I loved every minute of it."
text_example_negative = "This was the worst movie I have ever seen. Completely boring."

inputs_pos = tokenizer(text_example_positive, return_tensors="pt", padding=True, truncation=True)
inputs_neg = tokenizer(text_example_negative, return_tensors="pt", padding=True, truncation=True)


5. Test de prédiction sur une nouvelle phrase :


In [9]:
# Déplacer les inputs vers le même device que le modèle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs_pos = {k: v.to(device) for k, v in inputs_pos.items()}
inputs_neg = {k: v.to(device) for k, v in inputs_neg.items()}


with torch.no_grad():
    logits_pos = model(**inputs_pos).logits
    logits_neg = model(**inputs_neg).logits

predicted_class_id_pos = torch.argmax(logits_pos).item()
predicted_class_id_neg = torch.argmax(logits_neg).item()

print(f"Phrase: \"{text_example_positive}\" -> Prédiction: {model.config.id2label[predicted_class_id_pos]}")
print(f"Phrase: \"{text_example_negative}\" -> Prédiction: {model.config.id2label[predicted_class_id_neg]}")

Phrase: "This movie was absolutely fantastic! I loved every minute of it." -> Prédiction: POSITIVE
Phrase: "This was the worst movie I have ever seen. Completely boring." -> Prédiction: NEGATIVE


In [11]:
# si besoin de save le model
# model.save_pretrained("./my_sentiment_model")
# tokenizer.save_pretrained("./my_sentiment_model")

print("\nPreuve de concept terminée.")
#TODO est-ce necessaire de comparer la technique utilisée avec un autre ?


Preuve de concept terminée.
